In [25]:
import pandas as pd
import os
from typing import Optional
from pandas import DataFrame


In [26]:
from utils import copy_df
from config import DATA_FOLDER

### Download dos dados da página de orçamento da Prefeitura

In [27]:
BASE_URL = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/'

In [28]:
def build_url(base_url:str, file:str)->str:

    return base_url + file

def csv_fpath(fpath:str)->str:

    if not fpath.endswith('.csv'):
        fpath = fpath+'.csv'
    return fpath

def save_csv(df:pd.DataFrame, fpath:str, **save_kwargs)->str:

    fpath = csv_fpath(fpath)

    df.to_csv(fpath, sep=';', index=False, **save_kwargs)

    return os.path.abspath(fpath)


def load_csv(fpath:str, **read_kwargs)->pd.DataFrame:

    if not fpath.endswith('.csv'):
        raise ValueError(f'fpath {fpath} is not a csv file')
    
    return pd.read_csv(fpath, **read_kwargs)


def solve_data_file_path(file:str, folder=DATA_FOLDER)->str:

    if not os.path.exists(folder):
        os.mkdir(folder)

    folder = os.path.abspath(folder)

    if os.path.dirname(file) == folder:
        return file
    
    return os.path.join(folder, file)


def read_file_from_url(file:str, base_url=BASE_URL, **read_kwargs)->pd.DataFrame:

    url = build_url(base_url, file)

    return pd.read_csv(url, **read_kwargs)


In [29]:


def cached_download(file:str, folder= DATA_FOLDER, base_url = BASE_URL, 
                     read_params:Optional[dict]=None, save_params:Optional[dict]=None,
                     load_from_url_params:Optional[dict]=None)->pd.DataFrame:

    read_params = read_params or {}
    save_params = save_params or {}
    load_from_url_params = load_from_url_params or {}

    filepath = solve_data_file_path(file, folder)

    #early return if already saved - only reads
    if os.path.exists(filepath) and os.path.isfile(filepath):
        print(f'Loading file {file}')
        return load_csv(filepath, **read_params)
    

    df: DataFrame = read_file_from_url(file, base_url=base_url, **load_from_url_params)

    save_csv(df, filepath, **save_params)

    return df


In [30]:
file_execucao_2024= 'basedadosexecucao_1224.csv'
file_da_2024 = 'basedadosDA_1224.csv'


In [31]:
load_from_url_params = {'encoding' : 'latin1', 'sep' : ';'}
read_params = {'sep' : ';'}

df_execucao: DataFrame = cached_download(file_execucao_2024, 
                                         load_from_url_params=load_from_url_params,
                                         read_params=read_params)


df_da: DataFrame = cached_download(file_da_2024, 
                                         load_from_url_params=load_from_url_params,
                                        read_params=read_params)

Loading file basedadosexecucao_1224.csv
Loading file basedadosDA_1224.csv


/tmp/ipykernel_114107/3213725603.py:25: DtypeWarning: Columns (27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fpath, **read_kwargs)


### Execucao Orçamentária

In [32]:
df_execucao.head()

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
0,01/01/2024,31/12/2024,2024,2024,169108,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,1000,0,0,0,0,1000,18/01/2025
1,01/01/2024,31/12/2024,2024,2024,173723,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,"1317978,6",1314177,1314177,1314177,1314177,"3801,6",18/01/2025
2,01/01/2024,31/12/2024,2024,2024,171046,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,"3131324,4","3131324,4","3131324,4","2940879,9","2791266,85",0,18/01/2025
3,01/01/2024,31/12/2024,2024,2024,173862,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,"37258939,15","37222552,12","37222552,12","30999672,71","30291725,36","36387,03",18/01/2025
4,01/01/2024,31/12/2024,2024,2024,180471,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,2000000,2000000,2000000,2000000,2000000,0,18/01/2025


Como checamos abaixo, é a execução do ano inteiro de 2024.

In [33]:
df_execucao['DataInicial'].unique()

array(['01/01/2024'], dtype=object)

In [34]:
df_execucao['DataFinal'].unique()

array(['31/12/2024'], dtype=object)

In [35]:
df_execucao['Cd_AnoExecucao'].unique()

array([2024])

In [36]:
df_execucao['Cd_Exercicio'].unique()

array([2024])

Vamos filtrar as colunas que nos interessam

In [37]:
for col in df_execucao.columns:
    print(col)

DataInicial
DataFinal
Cd_AnoExecucao
Cd_Exercicio
Cd_Dotacao_Id
Administracao
Cd_Orgao
Sigla_Orgao
Ds_Orgao
Cd_Unidade
Ds_Unidade
Cd_Funcao
Ds_Funcao
Cd_SubFuncao
Ds_SubFuncao
Cd_Programa
Ds_Programa
PA
PAPA
ProjetoAtividade
Ds_Projeto_Atividade
ProjAtiv_Descricao
Cd_Despesa
Ds_Despesa
Categoria_Despesa
Ds_Categoria
Grupo_Despesa
Ds_Grupo
Cd_Modalidade
Ds_Modalidade
Cd_Elemento
Cd_Fonte
Ds_Fonte
COD_EX_FONT_REC
TXT_EX_FONT_REC
COD_DSTN_REC
TXT_DSTN_REC
COD_VINC_REC_PMSP
TXT_VINC_PMSP
COD_TIP_CRED_ORCM
TXT_TIP_CRED_ORCM
COD_RDZD_FONT_REC
TXT_FONT_REC_RDZD
Cd_Nro_Emenda_Dotacao
Vl_Orcado_Ano
Vl_Suplementado
Vl_Reduzido
Vl_SuplementadoLiquido
Vl_SuplementadoEmTramitacao
Vl_ReduzidoEmTramitacao
Vl_Orcado_Atualizado
Vl_Congelado
Vl_Descongelado
Vl_CongeladoLiquido
Disponivel
Vl_ReservadoLiquido
Vl_EmpenhadoLiquido
Vl_Liquidado
Vl_Pago
Saldo_Dotacao
DataExtracao


In [38]:
codigos_dotacao = [col for col in df_execucao.columns if (col.startswith('Cd')
                   or col in {'ProjetoAtividade', 'Categoria_Despesa'})
                   and col not in {'Cd_AnoExecucao', 'Cd_Exercicio', 'Cd_Dotacao_Id'}]

In [39]:
codigos_dotacao

['Cd_Orgao',
 'Cd_Unidade',
 'Cd_Funcao',
 'Cd_SubFuncao',
 'Cd_Programa',
 'ProjetoAtividade',
 'Cd_Despesa',
 'Categoria_Despesa',
 'Cd_Modalidade',
 'Cd_Elemento',
 'Cd_Fonte',
 'Cd_Nro_Emenda_Dotacao']

In [40]:
@copy_df
def build_dotacao(df:pd.DataFrame, cols_cod_dotacao:list[str]=codigos_dotacao)->DataFrame:

    df_cods = df[cols_cod_dotacao].copy()
    df['dotacao'] = df_cods.astype(str).agg('.'.join, axis=1)

    return df

In [41]:
df_execucao = build_dotacao(df_execucao)

In [42]:
df_execucao['dotacao'].sample(10)

4901     46.10.15.122.3024.2100.33900800.3.90.8.0.nan
8971    83.10.16.122.3024.2100.31901100.3.90.11.9.nan
1471    16.18.12.368.3010.2839.33503900.3.50.39.0.nan
3562    28.17.28.846.3004.6815.31901300.3.90.13.0.nan
2172    20.10.26.572.3009.1240.44905100.4.90.51.0.nan
3496    27.10.18.542.3005.6660.33903900.3.90.39.0.nan
5621    57.10.15.452.3022.1137.44903900.4.90.39.0.nan
1693    16.22.12.368.3010.2839.33503900.3.50.39.0.nan
5288    52.10.15.451.3022.1170.44903900.4.90.39.0.nan
4308    34.10.14.422.3018.4327.33503900.3.50.39.0.nan
Name: dotacao, dtype: object

In [43]:
df_execucao['dotacao'].str.split('.').apply(lambda x: len(x)).unique()

array([12, 13])

In [44]:
df_execucao['dotacao'].duplicated().any()

np.True_

In [45]:
df_execucao['dotacao'].duplicated().mean()

np.float64(0.4341823630136986)

In [46]:
colunas_descricao = [col for col in df_execucao.columns
                     if col.startswith('Ds')]

In [47]:
colunas_descricao

['Ds_Orgao',
 'Ds_Unidade',
 'Ds_Funcao',
 'Ds_SubFuncao',
 'Ds_Programa',
 'Ds_Projeto_Atividade',
 'Ds_Despesa',
 'Ds_Categoria',
 'Ds_Grupo',
 'Ds_Modalidade',
 'Ds_Fonte']

In [53]:
df_execucao.columns

Index(['DataInicial', 'DataFinal', 'Cd_AnoExecucao', 'Cd_Exercicio',
       'Cd_Dotacao_Id', 'Administracao', 'Cd_Orgao', 'Sigla_Orgao', 'Ds_Orgao',
       'Cd_Unidade', 'Ds_Unidade', 'Cd_Funcao', 'Ds_Funcao', 'Cd_SubFuncao',
       'Ds_SubFuncao', 'Cd_Programa', 'Ds_Programa', 'PA', 'PAPA',
       'ProjetoAtividade', 'Ds_Projeto_Atividade', 'ProjAtiv_Descricao',
       'Cd_Despesa', 'Ds_Despesa', 'Categoria_Despesa', 'Ds_Categoria',
       'Grupo_Despesa', 'Ds_Grupo', 'Cd_Modalidade', 'Ds_Modalidade',
       'Cd_Elemento', 'Cd_Fonte', 'Ds_Fonte', 'COD_EX_FONT_REC',
       'TXT_EX_FONT_REC', 'COD_DSTN_REC', 'TXT_DSTN_REC', 'COD_VINC_REC_PMSP',
       'TXT_VINC_PMSP', 'COD_TIP_CRED_ORCM', 'TXT_TIP_CRED_ORCM',
       'COD_RDZD_FONT_REC', 'TXT_FONT_REC_RDZD', 'Cd_Nro_Emenda_Dotacao',
       'Vl_Orcado_Ano', 'Vl_Suplementado', 'Vl_Reduzido',
       'Vl_SuplementadoLiquido', 'Vl_SuplementadoEmTramitacao',
       'Vl_ReduzidoEmTramitacao', 'Vl_Orcado_Atualizado', 'Vl_Congelado',
       'Vl_D

In [52]:
cols_valores = ['Vl_Orcado_Atualizado', 'Vl_EmpenhadoLiquido', 'Vl_CongeladoLiquido', 'Vl_Liquidado', 'Saldo_Dotacao']

In [54]:
cols_interesse = codigos_dotacao + colunas_descricao + cols_valores

In [55]:
df_execucao = df_execucao[cols_interesse]

In [57]:
df_execucao.head()

,Cd_Orgao,Cd_Unidade,Cd_Funcao,Cd_SubFuncao,Cd_Programa,ProjetoAtividade,Cd_Despesa,Categoria_Despesa,Cd_Modalidade,Cd_Elemento,...,Ds_Despesa,Ds_Categoria,Ds_Grupo,Ds_Modalidade,Ds_Fonte,Vl_Orcado_Atualizado,Vl_EmpenhadoLiquido,Vl_CongeladoLiquido,Vl_Liquidado,Saldo_Dotacao
0,7,10,8,244,3023,3399,44905100,4,90,51,...,Obras e Instalações,Despesas de Capital,INVESTIMENTOS,Aplicações Diretas,Tesouro Municipal - Recursos Vinculados,1000,0,0,0,1000
1,7,10,10,301,3003,1525,44505200,4,50,52,...,Equipamentos e Material Permanente,Despesas de Capital,INVESTIMENTOS,Transferências a Instituições Privadas sem fin...,Tesouro Municipal - Recursos Vinculados,"1317978,6",1314177,0,1314177,"3801,6"
2,7,10,10,301,3003,1525,44905100,4,90,51,...,Obras e Instalações,Despesas de Capital,INVESTIMENTOS,Aplicações Diretas,Tesouro Municipal - Recursos Vinculados,"3131324,4","3131324,4",0,"2940879,9",0
3,7,10,10,301,3003,1525,44905100,4,90,51,...,Obras e Instalações,Despesas de Capital,INVESTIMENTOS,Aplicações Diretas,Tesouro Municipal - Recursos Vinculados,"37258939,15","37222552,12",0,"30999672,71","36387,03"
4,7,10,10,301,3003,1525,44905100,4,90,51,...,Obras e Instalações,Despesas de Capital,INVESTIMENTOS,Aplicações Diretas,Alienação de Bens/Ativos,2000000,2000000,0,2000000,0
